In [1]:
import pandas as pd
import numpy as np
from weekly_predictions import run_weekly_predictions

In [2]:
# Data parameters
YEAR: int = 2024
WEEK: int = 3
MODEL: str = "clf"  # 'ensemble', 'stacking' or 'clf'

# Wagering parameters
WAGER_SIZE: int = 20
CONFIDENCE_THRESHOLD: float = 0.75

In [4]:
df = run_weekly_predictions(YEAR, WEEK, "clf")

In [5]:
df = df[
    [
        "away_team",
        "home_team",
        "predicted_winner",
        "win_probability",
        "away_moneyline",
        "home_moneyline",
    ]
]

In [6]:
df.head()

,away_team,home_team,predicted_winner,win_probability,away_moneyline,home_moneyline
32,NE,NYJ,NYJ,0.518386,245.0,-305.0
33,NYG,CLE,CLE,0.584910,270.0,-340.0
34,CHI,IND,IND,0.597042,NaN,NaN
35,HOU,MIN,MIN,0.655386,NaN,NaN
36,PHI,NO,PHI,0.573814,-142.0,120.0


In [7]:
# Function to calculate implied probability from moneyline
def implied_prob(moneyline):
    if moneyline > 0:
        return 100 / (moneyline + 100)
    else:
        return -moneyline / (-moneyline + 100)


# Function to calculate potential profit from a moneyline bet
def calculate_profit(moneyline, wager):
    if moneyline > 0:
        return wager * (moneyline / 100)
    else:
        return wager * (100 / -moneyline)


# Function to calculate expected profit
def expected_profit(win_probability, moneyline, wager):
    potential_payout = calculate_profit(moneyline, wager)
    return (win_probability * potential_payout) - ((1 - win_probability) * wager)

In [8]:
# Calculate implied probabilities for home and away teams from the moneyline
df["away_implied_prob"] = df["away_moneyline"].apply(implied_prob)
df["home_implied_prob"] = df["home_moneyline"].apply(implied_prob)

# Compare implied probability with model's predicted probability for value bets
df["away_value_bet"] = df["win_probability"] < df["away_implied_prob"]
df["home_value_bet"] = df["win_probability"] > df["home_implied_prob"]

# Calculate potential profit for away and home teams
df["away_potential_profit"] = df["away_moneyline"].apply(
    lambda x: calculate_profit(x, WAGER_SIZE)
)
df["home_potential_profit"] = df["home_moneyline"].apply(
    lambda x: calculate_profit(x, WAGER_SIZE)
)

# Calculate the projected profit based on the predicted winner (use home or away moneyline based on the prediction)
df["projected_profit"] = np.where(
    df["predicted_winner"] == df["away_team"],
    df["away_potential_profit"],  # Profit if away team wins
    df["home_potential_profit"],  # Profit if home team wins
)

In [9]:
display_columns = [
    "away_team",
    "home_team",
    "predicted_winner",
    "win_probability",
    "projected_profit",
    "away_moneyline",
    "home_moneyline",
    "away_implied_prob",
    "home_implied_prob",
    "away_value_bet",
    "home_value_bet",
    "away_potential_profit",
    "home_potential_profit",
]

In [10]:
# Display the top rows with the projected profit
df[display_columns].sort_values(by="projected_profit", ascending=False).head(10)

,away_team,home_team,predicted_winner,win_probability,projected_profit,away_moneyline,home_moneyline,away_implied_prob,home_implied_prob,away_value_bet,home_value_bet,away_potential_profit,home_potential_profit
47,WAS,CIN,WAS,0.776955,57.000000,285.0,-360.0,0.259740,0.782609,False,False,57.000000,5.555556
44,SF,LA,LA,0.546500,56.000000,-355.0,280.0,0.780220,0.263158,True,True,5.633803,56.000000
39,GB,TEN,GB,0.802335,21.000000,105.0,-125.0,0.487805,0.555556,False,True,21.000000,16.000000
43,BAL,DAL,BAL,0.738516,17.857143,-112.0,-108.0,0.528302,0.519231,False,True,17.857143,18.518519
37,LAC,PIT,PIT,0.564302,14.814815,114.0,-135.0,0.467290,0.574468,False,False,22.800000,14.814815
36,PHI,NO,PHI,0.573814,14.084507,-142.0,120.0,0.586777,0.454545,True,True,14.084507,24.000000
42,DET,ARI,DET,0.660283,12.048193,-166.0,140.0,0.624060,0.416667,False,True,12.048193,28.000000
45,KC,ATL,KC,0.705711,9.174312,-218.0,180.0,0.685535,0.357143,False,True,9.174312,36.000000
41,MIA,SEA,SEA,0.643499,7.194245,225.0,-278.0,0.307692,0.735450,False,False,45.000000,7.194245
46,JAX,BUF,BUF,0.773461,7.194245,225.0,-278.0,0.307692,0.735450,False,True,45.000000,7.194245


In [11]:
# Filter for win probability above threshold and value bets
filtered_df = (
    df.loc[(df["win_probability"] > CONFIDENCE_THRESHOLD), display_columns]
    .sort_values(by="projected_profit", ascending=False)
    .head(10)
)


display(filtered_df)

,away_team,home_team,predicted_winner,win_probability,projected_profit,away_moneyline,home_moneyline,away_implied_prob,home_implied_prob,away_value_bet,home_value_bet,away_potential_profit,home_potential_profit
47,WAS,CIN,WAS,0.776955,57.000000,285.0,-360.0,0.259740,0.782609,False,False,57.0,5.555556
39,GB,TEN,GB,0.802335,21.000000,105.0,-125.0,0.487805,0.555556,False,True,21.0,16.000000
46,JAX,BUF,BUF,0.773461,7.194245,225.0,-278.0,0.307692,0.735450,False,True,45.0,7.194245
38,DEN,TB,TB,0.830236,6.153846,260.0,-325.0,0.277778,0.764706,False,True,52.0,6.153846


In [12]:
# Bets Taken So Far
TEAMS_BET = ["NO", "PHI", "NE", "KC", "BUF"]

# Game Rows
game_rows = df.loc[
    (df["away_team"].isin(TEAMS_BET) | df["home_team"].isin(TEAMS_BET)), display_columns
]
display(game_rows)

,away_team,home_team,predicted_winner,win_probability,projected_profit,away_moneyline,home_moneyline,away_implied_prob,home_implied_prob,away_value_bet,home_value_bet,away_potential_profit,home_potential_profit
32,NE,NYJ,NYJ,0.518386,6.557377,245.0,-305.0,0.289855,0.753086,False,False,49.000000,6.557377
36,PHI,NO,PHI,0.573814,14.084507,-142.0,120.0,0.586777,0.454545,True,True,14.084507,24.000000
45,KC,ATL,KC,0.705711,9.174312,-218.0,180.0,0.685535,0.357143,False,True,9.174312,36.000000
46,JAX,BUF,BUF,0.773461,7.194245,225.0,-278.0,0.307692,0.735450,False,True,45.000000,7.194245
